# Fitting gaus and finding peaks of the maps

This notebook is preliminary to the fitting script file.

It's aim is to:

- If any of the cl maps need to be cropped beforehand
- To know the ideal INITIAL fitting parameters

It also:
- Finds peaks (for the position, height and width parameters)
- Creates a spectral stack


Import packages:

In [1]:
%matplotlib qt
#For pop-up window plots, with interactive functionality. If error, use instead %matplotlib tk 
import lumispy as lum
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
#sns.set(style='darkgrid')

## Loading HYPCard files

Use the `load_hypcard` function to load the AttoLight SEM specific files. State the relative path to the `HYPCard.bin` file.

*You can also leave the path empty. A pop-up window will appear to select the `HYPCard.bin` file from the browser.*

In [2]:
# PARAMS
root = os.path.abspath('G:\My Drive\PhD')
session = os.path.relpath('projects/external_measurements/202103_Linsong_YuQi/20210322_CL_YuQi_samples')

extension = '*\*.hspy'
####
# GO TO AUTOMATISED TO GET GENERAL PROCESSED DATA
import os, glob
folder = os.path.join(root, session)
session_path = os.path.join(root, session, extension)
# For HYPMaps
paths_hypmap = [p for p in glob.glob(session_path, recursive=True) if 'fit' not in p]
paths_hypmap.sort()

[('i={}'.format(i), os.path.split(os.path.dirname(f))[-1]) for i,f in enumerate(paths_hypmap)]

[('i=0', 'HYP-MAP1'),
 ('i=1', 'HYP-MAP1'),
 ('i=2', 'HYP-MAP2'),
 ('i=3', 'HYP-MAP2'),
 ('i=4', 'HYP-MAP3'),
 ('i=5', 'HYP-MAP4'),
 ('i=6', 'HYP-MAP5'),
 ('i=7', 'HYP-MAP6'),
 ('i=8', 'HYP-MAP7')]

In [4]:
cls = hs.load(paths_hypmap, signal_type='CL_SEM')
print(cls)

[<CLSEMSpectrum, title: HYP-MAP5, dimensions: (118, 118|1022)>, <CLSEMSpectrum, title: HYP-MAP6, dimensions: (118, 118|1022)>]


In [6]:
cls[1].plot()

## Select which maps need cropping before fitting

Select `ith` file and if only a smaller region of the map needs to be fitted.

If the map has been cropped, save the cropped_map in the folder as `*processed_roi.hspy`
The script will prioritize the cropped map file over the original sized one.

In [11]:
#########   PARAMS   ######
i = 1
crop_roi = False

##########################

cl = cls[i]
if crop_roi:
    roi = hs.roi.RectangularROI(0,0,10,10)
    cl.plot()
    roi.interactive(cl)

In [12]:
if crop_roi:
    cl = roi(cl)
    path_crop = os.path.join(os.path.dirname(paths_hypmap[i]),  'HYPCard_processed_cropped.hspy')
    cl.save(path_crop)

Please answer y or n.


# Get the best initial parameters

Get the fit for each map:
Change the `parameters` within the `fit_3_gausians` function

In [13]:
cl.sum().plot()

In [17]:
# def fit_3_gaussians(cl_object):
def fit_2_gaussians(cl_object):

    # Create model (with 2 Gaussian)
    m = cl_object.create_model()

    g_pvk = hs.model.components1D.Expression(
        expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
        name="Perovskite",
        position="x0",
        height=1,
        fwhm=1,
        x0=1,
        module="numpy")

    g_pbi2 = hs.model.components1D.Expression(
        expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
        name="PbI2",
        position="x0",
        height=1,
        fwhm=1,
        x0=1,
        module="numpy")

    # g_decay = hs.model.components1D.Expression(
    #     expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
    #     name="Decay peak",
    #     position="x0",
    #     height=1,
    #     fwhm=1,
    #     x0=1,
    #     module="numpy")

    bkg = hs.model.components1D.Offset()

    #m.extend([g_pvk, g_pbi2, g_decay, bkg])
    m.extend([g_pvk, g_pbi2, bkg])

    # EDIT HERE
    ######################
    #COPY HERE
    #Set limits to the gausians
    g_pvk.x0.value = 750
    g_pvk.x0.bmin, g_pvk.x0.bmax = 690, 810
    g_pvk.height.value, g_pvk.height.bmin, g_pvk.height.bmax = 10, -10, 500
    g_pvk.fwhm.value, g_pvk.fwhm.bmin, g_pvk.fwhm.bmax = 20, 0, 100
    
    g_pbi2.x0.value = 510
    g_pbi2.x0.bmin, g_pbi2.x0.bmax = 470, 550
    g_pbi2.height.value, g_pbi2.height.bmin, g_pbi2.height.bmax = 10, -10, 500
    g_pbi2.fwhm.value, g_pbi2.fwhm.bmin, g_pbi2.fwhm.bmax = 20, 0, 100
    
    bkg.offset.value = 0
    
    # g_decay.x0.value = 650
    # g_decay.x0.bmin, g_decay.x0.bmax = 510, 750
    # g_decay.height.value, g_decay.height.bmin, g_decay.height.bmax = 10, -10, 100
    # g_decay.fwhm.value, g_decay.fwhm.bmin, g_decay.fwhm.bmax = 20, 0, 100
    #END COPY HERE
    ######################
    
    
    #Fit for all the positions
    m.fit(bounded=True)

    # Plot
    import matplotlib.pyplot as plt
    m.plot(plot_components=True)

    return m

# fit_3_gaussians(cl)
fit_2_gaussians(cl)

<Model1D, title: HYP-MAP6>

# RUN SCRIPT NOW

In the script, you will need to update the fitting values to the ideal initial conditions found


# Post-script code

Use this code to load the fitted models and plot them.

In [35]:
extension = "*fitted.hspy"

import os, glob
folder = os.path.join(root, session, 'fit_imgs')
session_path = os.path.join(folder, extension)

# For HYPMaps
paths_fits = [p for p in glob.glob(session_path, recursive=True)]
paths_fits.sort()
[('i={}'.format(i), os.path.basename(f)) for i,f in enumerate(paths_fits)]

[('i=0', 'HYP-MAP5_binned_22_2_fitted.hspy'),
 ('i=1', 'HYP-MAP5_fitted.hspy'),
 ('i=2', 'HYP-MAP6_binned_22_2_fitted.hspy'),
 ('i=3', 'HYP-MAP6_fitted.hspy')]

In [40]:
for f in paths_fits[:]:
    cl = hs.load(f, signal_type='CL_SEM')
    m = cl.models.restore("gaus_fit")
    #print(m.components)
    
    comps = [m.components.Perovskite.x0.as_signal(),
         m.components.Perovskite.height.as_signal(),
         m.components.Perovskite.fwhm.as_signal(),]

    hs.plot.plot_images(comps, axes_decor='off',
                        scalebar=[0], cmap=['nipy_spectral', 'viridis', 'inferno'],
                        vmin=[780,0,0])
    plt.tight_layout()
